In [1]:
# Adjust Python path within the notebook
import sys
project_root = '/Users/thangnguyen/Documents/GitHub/project-1-individual-knam2609'
if project_root not in sys.path:
    sys.path.insert(0, project_root)

import scripts

In [2]:
from pyspark.sql import SparkSession

# Create a spark session
spark = (
    SparkSession.builder.appName("Project 1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

24/08/21 19:03:33 WARN Utils: Your hostname, THANGs-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 100.86.89.69 instead (on interface en0)
24/08/21 19:03:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/21 19:03:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
sdf_hv = spark.read.parquet('../data/raw/high_volume/2023/2023-03.parquet')
sdf_hv.show(1, vertical=True, truncate=100)

-RECORD 0-----------------------------------
 hvfhs_license_num    | HV0003              
 dispatching_base_num | B03404              
 originating_base_num | B03404              
 request_datetime     | 2023-03-01 00:02:05 
 on_scene_datetime    | 2023-03-01 00:05:26 
 pickup_datetime      | 2023-03-01 00:05:26 
 dropoff_datetime     | 2023-03-01 00:11:05 
 PULocationID         | 114                 
 DOLocationID         | 79                  
 trip_miles           | 1.21                
 trip_time            | 339                 
 base_passenger_fare  | 7.72                
 tolls                | 0.0                 
 bcf                  | 0.23                
 sales_tax            | 0.69                
 congestion_surcharge | 2.75                
 airport_fee          | 0.0                 
 tips                 | 0.0                 
 driver_pay           | 5.5                 
 shared_request_flag  | N                   
 shared_match_flag    | N                   
 access_a_

In [4]:
sdf_yellow = spark.read.parquet('../data/raw/yellow/2023/2023-03.parquet')
sdf_yellow.show(1, vertical=True, truncate=100)

-RECORD 0------------------------------------
 VendorID              | 2                   
 tpep_pickup_datetime  | 2023-03-01 00:06:43 
 tpep_dropoff_datetime | 2023-03-01 00:16:43 
 passenger_count       | 1                   
 trip_distance         | 0.0                 
 RatecodeID            | 1                   
 store_and_fwd_flag    | N                   
 PULocationID          | 238                 
 DOLocationID          | 42                  
 payment_type          | 2                   
 fare_amount           | 8.6                 
 extra                 | 1.0                 
 mta_tax               | 0.5                 
 tip_amount            | 0.0                 
 tolls_amount          | 0.0                 
 improvement_surcharge | 1.0                 
 total_amount          | 11.1                
 congestion_surcharge  | 0.0                 
 Airport_fee           | 0.0                 
only showing top 1 row



In [5]:
sdf_hv.count()

20413539

In [6]:
sdf_yellow.filter(sdf_yellow["mta_tax"] > 0).select(["payment_type"]).distinct()

payment_type
0
1
3
2
4


In [7]:
sdf_yellow.select("mta_tax").distinct()

mta_tax
0.0
3.5
0.8
4.0
0.5
3.0
-0.5


In [8]:
sdf_yellow.count()

3403766

In [9]:
sdf_yellow.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [10]:
sdf_hv.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- originating_base_num: string (nullable = true)
 |-- request_datetime: timestamp_ntz (nullable = true)
 |-- on_scene_datetime: timestamp_ntz (nullable = true)
 |-- pickup_datetime: timestamp_ntz (nullable = true)
 |-- dropoff_datetime: timestamp_ntz (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- trip_time: long (nullable = true)
 |-- base_passenger_fare: double (nullable = true)
 |-- tolls: double (nullable = true)
 |-- bcf: double (nullable = true)
 |-- sales_tax: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- tips: double (nullable = true)
 |-- driver_pay: double (nullable = true)
 |-- shared_request_flag: string (nullable = true)
 |-- shared_match_flag: string (nullable = true)
 |-- access_a_

In [11]:
from pyspark.sql.functions import col, count, when

# Check for actual nulls and empty strings
null_or_empty = sdf_hv.select([
    count(when(col(c).isNull(), c)).alias(c) for c in sdf_hv.columns
])

null_or_empty


24/08/21 19:03:46 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,trip_time,base_passenger_fare,tolls,bcf,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
0,0,5851726,0,5851726,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [12]:
from pyspark.sql.functions import col, count, when

# Check for actual nulls and empty strings
null_or_empty = sdf_yellow.select([
    count(when(col(c).isNull(), c)).alias(c) for c in sdf_yellow.columns
])

null_or_empty


VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,0,0,87619,0,87619,87619,0,0,0,0,0,0,0,0,0,0,87619,87619


In [13]:
unique_values_df = sdf_hv.select("access_a_ride_flag").distinct()
unique_values_df 

access_a_ride_flag
N
""


In [14]:
sdf_hv.filter(sdf_hv["wav_request_flag"] == 'Y').select("originating_base_num").distinct()

originating_base_num
B00887
B03404
B02026
B03153
B03406


In [15]:
type(spark)

pyspark.sql.session.SparkSession

In [16]:
# Drop rows where all columns are null
sdf_hv = sdf_hv.na.drop()

In [17]:
sdf_hv.count()

14561813

In [18]:
from pyspark.sql.types import TimestampNTZType
from datetime import datetime

#start_time = datetime.strptime("2023-03-01 00:00:00", "%Y-%m-%d %H:%M:%S")
#end_time = datetime.strptime("2023-03-31 23:59:59", "%Y-%m-%d %H:%M:%S")
#print(type(start_time))

start_time = "2023-03-01 00:00:00"
end_time = "2023-03-31 23:00:00"

# Find all datetime columns
datetime_columns = [f.name for f in sdf_hv.schema.fields if isinstance(f.dataType, TimestampNTZType)]
df = sdf_hv
# Filter out the date range for each datetime column using correct function
for column in datetime_columns:
    print(column)
    df = df.filter(~col(column).between(start_time, end_time))
    print(f"Filtered data for column {column}:")

df

request_datetime
Filtered data for column request_datetime:
on_scene_datetime
Filtered data for column on_scene_datetime:
pickup_datetime
Filtered data for column pickup_datetime:
dropoff_datetime
Filtered data for column dropoff_datetime:


hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,trip_time,base_passenger_fare,tolls,bcf,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
HV0003,B03404,B03404,2023-03-31 23:07:35,2023-03-31 23:08:18,2023-03-31 23:09:53,2023-03-31 23:22:50,4,170,2.29,777,15.4,0.0,0.46,1.37,2.75,0.0,0.0,14.32,N,N,,N,N
HV0003,B03404,B03404,2023-03-31 23:22:37,2023-03-31 23:25:05,2023-03-31 23:27:06,2023-03-31 23:42:13,170,249,2.18,907,18.67,0.0,0.56,1.66,2.75,0.0,0.0,15.4,N,N,,N,N
HV0003,B03404,B03404,2023-03-31 23:39:03,2023-03-31 23:45:30,2023-03-31 23:45:42,2023-03-31 23:55:43,114,170,2.17,601,14.75,0.0,0.44,1.31,2.75,0.0,0.0,12.51,N,N,,N,N
HV0003,B03404,B03404,2023-03-31 23:55:57,2023-03-31 23:57:26,2023-03-31 23:58:05,2023-04-01 00:33:17,162,265,16.85,2112,50.03,0.0,1.5,4.44,2.75,0.0,5.87,49.13,N,N,,N,N
HV0003,B03404,B03404,2023-03-31 23:03:44,2023-03-31 23:14:08,2023-03-31 23:15:00,2023-03-31 23:38:30,148,17,4.12,1410,41.72,0.0,1.25,3.7,2.75,0.0,0.0,28.5,N,N,,N,N
HV0003,B03404,B03404,2023-03-31 23:42:29,2023-03-31 23:45:20,2023-03-31 23:50:20,2023-04-01 00:12:34,17,79,4.97,1334,56.43,0.0,1.69,5.01,2.75,0.0,0.0,43.05,N,N,,N,N
HV0003,B03404,B03404,2023-03-31 23:38:12,2023-03-31 23:41:44,2023-03-31 23:43:18,2023-03-31 23:53:47,76,222,2.47,629,12.46,0.0,0.37,1.11,0.0,0.0,0.0,9.16,N,N,,N,N
HV0003,B03404,B03404,2023-03-31 23:15:22,2023-03-31 23:16:38,2023-03-31 23:18:37,2023-03-31 23:29:44,50,158,1.75,667,12.4,0.0,0.37,1.1,2.75,0.0,0.0,8.56,N,N,,N,N
HV0003,B03404,B03404,2023-03-31 23:27:36,2023-03-31 23:32:03,2023-03-31 23:32:03,2023-04-01 00:18:50,158,254,15.61,2807,51.69,0.0,1.55,4.59,2.75,0.0,0.0,46.89,N,N,,N,N
HV0003,B03404,B03404,2023-03-31 23:37:26,2023-03-31 23:43:43,2023-03-31 23:44:28,2023-04-01 00:12:20,138,142,9.68,1672,44.82,0.0,1.34,3.98,2.75,0.0,0.0,30.09,N,N,,N,N


In [19]:
time = datetime.strptime("2023-03-01 13:03:47.000", "%Y-%m-%d %H:%M:%S.%f")

In [20]:
type(time)

datetime.datetime

In [21]:
print(datetime_columns)

['request_datetime', 'on_scene_datetime', 'pickup_datetime', 'dropoff_datetime']


In [22]:
df.count()

30162

In [23]:
taxi_zones = spark.read.csv("../data/raw/taxi_zones.csv", header=True, inferSchema=True)

In [24]:
taxi_zones

LocationID,Borough,Zone,service_zone
1,EWR,Newark Airport,EWR
2,Queens,Jamaica Bay,Boro Zone
3,Bronx,Allerton/Pelham G...,Boro Zone
4,Manhattan,Alphabet City,Yellow Zone
5,Staten Island,Arden Heights,Boro Zone
6,Staten Island,Arrochar/Fort Wad...,Boro Zone
7,Queens,Astoria,Boro Zone
8,Queens,Astoria Park,Boro Zone
9,Queens,Auburndale,Boro Zone
10,Queens,Baisley Park,Boro Zone


In [25]:
yellow_taxi_zones = taxi_zones.filter(taxi_zones["service_zone"]== "Yellow Zone")
yellow_zones_ID = set([row['LocationID'] for row in yellow_taxi_zones.select("LocationID").distinct().collect()])

In [26]:
138 in yellow_zones_ID

False

In [27]:
sdf_yellow.filter(~(sdf_yellow["PUlocationID"].isin(yellow_zones_ID)))

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
2,2023-03-01 00:08:25,2023-03-01 00:39:30,2,12.4,1,N,138,231,1,52.7,6.0,0.5,12.54,0.0,1.0,76.49,2.5,1.25
2,2023-03-01 00:29:18,2023-03-01 01:06:01,2,21.03,2,N,132,239,1,70.0,0.0,0.5,12.0,6.55,1.0,93.8,2.5,1.25
1,2023-03-01 00:50:37,2023-03-01 01:04:46,1,6.4,1,N,132,258,1,26.8,2.25,0.5,6.1,0.0,1.0,36.65,0.0,1.25
2,2023-03-01 00:25:51,2023-03-01 00:55:03,1,9.5,1,N,138,10,1,42.2,6.0,0.5,50.95,0.0,1.0,101.9,0.0,1.25
2,2023-03-01 00:19:08,2023-03-01 00:36:39,1,9.39,1,N,138,230,1,37.3,6.0,0.5,13.78,6.55,1.0,68.88,2.5,1.25
2,2023-03-01 00:14:55,2023-03-01 00:50:55,1,19.68,2,N,132,158,1,70.0,0.0,0.5,16.36,6.55,1.0,98.16,2.5,1.25
1,2023-03-01 00:29:34,2023-03-01 01:10:50,1,6.5,99,N,76,262,1,53.5,0.0,0.5,0.0,0.0,1.0,55.0,0.0,0.0
2,2023-03-01 00:30:58,2023-03-01 01:03:39,1,19.35,2,N,132,152,1,70.0,0.0,0.5,11.9,6.55,1.0,91.2,0.0,1.25
1,2023-03-01 00:54:35,2023-03-01 01:26:09,1,21.3,2,N,132,87,1,70.0,3.75,0.5,15.05,0.0,1.0,90.3,2.5,1.25
1,2023-03-01 00:02:15,2023-03-01 00:33:46,1,9.8,1,N,138,61,1,45.0,7.25,0.5,10.75,0.0,1.0,64.5,0.0,1.25


In [ ]:
sdf_yellow.count()

In [ ]:
import os
os.path.basename("../data/raw/yellow/2023/2023-03.parquet")

In [ ]:
directory_path = "../data/raw/yellow/2023"
[os.path.join(directory_path, f) for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f))]


In [29]:
test_df = scripts.clean_yellow.clean_directory_data("../data/raw/yellow/2023", "", taxi_zones)

24/08/21 19:05:00 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


NotADirectoryError: [Errno 20] Not a directory: '../data/raw/yellow/2023/2023-03.parquet'